In [2]:
import boto3

In [13]:
def create_movie_table(dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.create_table(
        TableName='Movies',
        KeySchema=[
            {
                'AttributeName': 'year',
                'KeyType': 'HASH'  # Partition key
            },
            {
                'AttributeName': 'title',
                'KeyType': 'RANGE'  # Sort key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'year',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'title',
                'AttributeType': 'S'
            },

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
    return table

In [15]:
movie_table = create_movie_table()

In [16]:
def put_movie(title, year, plot, rating, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('Movies')
    response = table.put_item(
       Item={
            'year': year,
            'title': title,
            'info': {
                'plot': plot,
                'rating': rating
            }
        }
    )
    return response

In [17]:
movie_resp = put_movie("The Big New Movie", 2015,
                           "Nothing happens at all.", 0)

In [19]:
from botocore.exceptions import ClientError

In [20]:
def get_movie(title, year, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('Movies')

    try:
        response = table.get_item(Key={'year': year, 'title': title})
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:
        return response['Item']

In [22]:
movie = get_movie("The Big New Movie", 2015,)
if movie:
    print("Get movie succeeded:")
    print(movie)

Get movie succeeded:
{'year': Decimal('2015'), 'info': {'rating': Decimal('0'), 'plot': 'Nothing happens at all.'}, 'title': 'The Big New Movie'}


In [23]:
from decimal import Decimal

In [24]:
def update_movie(title, year, rating, plot, actors, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('Movies')

    response = table.update_item(
        Key={
            'year': year,
            'title': title
        },
        UpdateExpression="set info.rating=:r, info.plot=:p, info.actors=:a",
        ExpressionAttributeValues={
            ':r': Decimal(rating),
            ':p': plot,
            ':a': actors
        },
        ReturnValues="UPDATED_NEW"
    )
    return response

In [25]:
update_response = update_movie(
        "The Big New Movie", 2015, 5.5, "Everything happens all at once.",
        ["Larry", "Moe", "Curly"])
print("Update movie succeeded:")
print(update_response)

Update movie succeeded:
{'Attributes': {'info': {'actors': ['Larry', 'Moe', 'Curly'], 'plot': 'Everything happens all at once.', 'rating': Decimal('5.5')}}, 'ResponseMetadata': {'RequestId': '85H3BDO4SOF3UQ0VS8IMTU1VI7VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Tue, 06 Oct 2020 12:55:19 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '156', 'connection': 'keep-alive', 'x-amzn-requestid': '85H3BDO4SOF3UQ0VS8IMTU1VI7VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '3767510606'}, 'RetryAttempts': 0}}
